In [1]:
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

df = pd.read_csv('Raotbl6.csv')
df['date'] = pd.to_datetime(df['date'])
df.index = df['date']
df.drop('date', axis=1, inplace=True)

# make future targets
for i in range(12):
    df['rgnp_{}'.format(i)] = df['rgnp'].shift(-i-1)

df.dropna(inplace=True)

targets = [item for item in df.columns if 'rgnp_' in item]

X_train = df.drop(targets, axis=1)[: int(len(df) * 0.8)]
y_train = df[targets][: int(len(df) * 0.8)]

X_test = df.drop(targets, axis=1)[int(len(df) * 0.8) :]
y_test = df[targets][int(len(df) * 0.8) :]


In [2]:
class Raotbl6Dataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        
        
    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        X = torch.Tensor(self.X_train.iloc[idx].values)
        y = torch.Tensor(self.y_train.iloc[idx].values)
        return X, y

In [3]:
class Model(nn.Module):
    def __init__(self, in_feats, out_feats=12, hidden_units=32):
        super(Model, self).__init__()
        
        # 输入：(batch_size, z_dim, 1, 1)
        self.net = nn.Sequential(
            nn.Linear(in_feats, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, out_feats)
        )
    
    def forward(self, x):
        return self.net(x)

In [4]:
NUM_EPOCHS = 500
LEARNING_RATE = 2e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'



dataset = Raotbl6Dataset(X_train, y_train)
dataloader = DataLoader(dataset,batch_size=32, shuffle=True)
criterion = torch.nn. MSELoss()

model = Model(8).to(device)
opt = optim.Adam(model.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))

In [5]:

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, (X, y) in enumerate(dataloader):
        
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = criterion(y, pred)
        
        model.zero_grad()
        loss.backward()
        opt.step()
    print(
        f"Epoch [{epoch}/{NUM_EPOCHS}] \
          ERROR: {loss:.4f}"
    )


Epoch [0/500]           ERROR: 5665249.0000
Epoch [1/500]           ERROR: 6766489.5000
Epoch [2/500]           ERROR: 6046984.0000
Epoch [3/500]           ERROR: 6704573.0000
Epoch [4/500]           ERROR: 6238314.0000
Epoch [5/500]           ERROR: 6090670.5000
Epoch [6/500]           ERROR: 6996567.0000
Epoch [7/500]           ERROR: 5883912.0000
Epoch [8/500]           ERROR: 6255249.0000
Epoch [9/500]           ERROR: 6302207.0000
Epoch [10/500]           ERROR: 6677468.0000
Epoch [11/500]           ERROR: 5846011.5000
Epoch [12/500]           ERROR: 5714164.5000
Epoch [13/500]           ERROR: 6616586.5000
Epoch [14/500]           ERROR: 5457780.5000
Epoch [15/500]           ERROR: 6271841.0000
Epoch [16/500]           ERROR: 6062314.5000
Epoch [17/500]           ERROR: 5327139.0000
Epoch [18/500]           ERROR: 5645721.5000
Epoch [19/500]           ERROR: 5864180.5000
Epoch [20/500]           ERROR: 6147806.5000
Epoch [21/500]           ERROR: 5998652.5000
Epoch [22/500]      

Epoch [184/500]           ERROR: 1145074.3750
Epoch [185/500]           ERROR: 1200663.1250
Epoch [186/500]           ERROR: 1027273.3125
Epoch [187/500]           ERROR: 1017164.0000
Epoch [188/500]           ERROR: 1138254.0000
Epoch [189/500]           ERROR: 1067482.8750
Epoch [190/500]           ERROR: 1040549.4375
Epoch [191/500]           ERROR: 981044.6875
Epoch [192/500]           ERROR: 954344.1250
Epoch [193/500]           ERROR: 974051.2500
Epoch [194/500]           ERROR: 934773.6875
Epoch [195/500]           ERROR: 896630.0000
Epoch [196/500]           ERROR: 864238.3125
Epoch [197/500]           ERROR: 842936.0000
Epoch [198/500]           ERROR: 793079.0625
Epoch [199/500]           ERROR: 906532.8125
Epoch [200/500]           ERROR: 803527.5625
Epoch [201/500]           ERROR: 715678.0000
Epoch [202/500]           ERROR: 780468.0625
Epoch [203/500]           ERROR: 760718.8750
Epoch [204/500]           ERROR: 705542.8750
Epoch [205/500]           ERROR: 770902.8125
Epo

Epoch [368/500]           ERROR: 19006.8633
Epoch [369/500]           ERROR: 18354.0625
Epoch [370/500]           ERROR: 15863.4668
Epoch [371/500]           ERROR: 15994.8125
Epoch [372/500]           ERROR: 18140.0410
Epoch [373/500]           ERROR: 16175.2783
Epoch [374/500]           ERROR: 15778.8213
Epoch [375/500]           ERROR: 17274.9688
Epoch [376/500]           ERROR: 18792.8652
Epoch [377/500]           ERROR: 16362.4932
Epoch [378/500]           ERROR: 15920.2881
Epoch [379/500]           ERROR: 18069.3418
Epoch [380/500]           ERROR: 16469.9863
Epoch [381/500]           ERROR: 17217.3652
Epoch [382/500]           ERROR: 18992.1035
Epoch [383/500]           ERROR: 16638.7578
Epoch [384/500]           ERROR: 13901.7793
Epoch [385/500]           ERROR: 17286.6387
Epoch [386/500]           ERROR: 17399.2402
Epoch [387/500]           ERROR: 14857.7471
Epoch [388/500]           ERROR: 16751.0742
Epoch [389/500]           ERROR: 16263.5508
Epoch [390/500]           ERROR:

In [6]:
pred = pd.DataFrame(model(torch.Tensor(X_test.values).to(device)), columns=y_test.columns, index=y_test.index)

rgnp_0     160.920304
rgnp_1     187.063986
rgnp_2     185.027259
rgnp_3     172.836946
rgnp_4     157.783776
rgnp_5     121.775486
rgnp_6     124.746922
rgnp_7     139.684623
rgnp_8      76.782863
rgnp_9      63.746731
rgnp_10     60.439411
rgnp_11     54.715396
dtype: float64

In [ ]:
        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
        disc_real = disc(real).reshape(-1)
        # 设置real=1, fake=0
        # disc(real)的target是[1,1,1,1,...]
        loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
        
        
        disc_fake = disc(fake.detach()).reshape(-1)
        # disc(fake)的target是(0,0,0,0,...)
        loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        
        # 损失是两者的均值
        loss_disc = (loss_disc_real + loss_disc_fake) / 2
        

        
        ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
        output = disc(fake).reshape(-1)
        # fake希望disc(fake)预测的结果=1
        loss_gen = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()
        # 每100个batch记录一次模型效果
        if count % 100 == 0:
            print(
                f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(dataloader)} \
                  Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}"
            )
            
            with torch.no_grad():
                fake = gen(fixed_noise)
                # take out (up to) 32 examples
                img_grid_real = torchvision.utils.make_grid(
                    real[:32], normalize=True
                )
                img_grid_fake = torchvision.utils.make_grid(
                    fake[:32], normalize=True
                )

                writer_real.add_image("Real", img_grid_real, global_step=step)
                writer_fake.add_image("Fake", img_grid_fake, global_step=step)
                writer_real.flush()
                writer_fake.flush()
            step += 1
        count += 1